# BigQuery ML

In [ ]:
service_account_file='../../service-account.json'

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=service_account_file
%load_ext google.cloud.bigquery
import json
with open(service_account_file, 'r') as f:
    sa = json.load(f)
project_id=sa['project_id']
os.environ['GOOGLE_CLOUD_PROJECT']=project_id

In [ ]:
%load_ext google.cloud.bigquery

## Create the Model

In [ ]:
%%bigquery

#%sql -d standard

CREATE OR REPLACE MODEL `bqml_tutorial.sample_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'

## Evaluate the Model

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))

## Make Predictions

In [ ]:
%%bigquery

SELECT
  country,
  SUM(predicted_label) as total_predicted_purchases
FROM
  ml.PREDICT(MODEL `bqml_tutorial.sample_model`, (
SELECT
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(totals.pageviews, 0) AS pageviews,
  IFNULL(geoNetwork.country, "") AS country
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
GROUP BY country
ORDER BY total_predicted_purchases DESC
LIMIT 10;

In [ ]:
%%bigquery 
select * from ML.FEATURE_INFO(MODEL `bqml_tutorial.sample_model`)

In [ ]:
%%bigquery 
select * from ML.TRAINING_INFO(MODEL `bqml_tutorial.sample_model`)

In [ ]:
%%bigquery 
select * from ML.WEIGHTS(MODEL `bqml_tutorial.sample_model`)

In [ ]:
%%bigquery 
select * from ML.CENTROIDS(MODEL `bqml_tutorial.sample_model`)